In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# cd drive/MyDrive/ECS271_project/house-price-prediction


In [2]:
%load_ext autoreload
%autoreload 2
from helpers import *
from networks import *
from tqdm import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

### Define some tunable variables

In [3]:
batch_size = 5
lr = 0.001
mom = 0.5
epochs = 50
networ_type = 'CNN' #FNN CNN
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [4]:
device

device(type='mps')

### Prepare data for training

In [5]:
train = MyDataset("train.csv", 0, 0.7)
test = MyDataset("train.csv", 0.7, 1)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

In [6]:
num_features = train.x.shape[1]
num_features

304

### Build the neural network

In [7]:
net = eval(networ_type)()
net.to(device)
net

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=85264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)

### Train

In [8]:
# criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=mom)

In [9]:
los = []
tlos = []
for epoch in range(epochs):
    tot_loss = 0.0
    net.train()
    for i, (inputs, labels) in enumerate(tqdm(train_loader), 0):
        inputs, labels = inputs.to(device), labels.to(device) # Move to GPU / CPU
        optimizer.zero_grad() # zero the parameter gradients

        labels = labels.unsqueeze(-1).unsqueeze(-1)
        # forward + backward + optimize
        l = [m.outer(m) for m in inputs]
        inputs = torch.empty((len(l),)+l[0].shape, dtype=torch.float32, device=device)
        for j in range(len(l)):
            inputs[j] = l[j]
        # for inp in inputs:
        # inputs = inputs.outer(inputs)  #np.array([np.diag(i) for i in inputs])
        inputs = inputs.unsqueeze(1) #.to_sparse()
        # print(inputs.shape)
        outputs = net(inputs).unsqueeze(-1)
        # print(outputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        tot_loss += loss.item()
    print(tot_loss)
    # ttot_loss = 0.0
    # net.eval()
    # for i, (inputs, labels) in enumerate(test_loader, 0):
    #     # inputs, labels = inputs.to(device), labels.to(device) # Move to GPU / CPU
    #     with torch.no_grad():
    #         labels = labels.reshape((-1, 1))
    #         inputs = torch.Tensor(inputs).unsqueeze(1) #.to_sparse()
    #         outputs = net(inputs)
    #         loss = criterion(outputs, labels)
    #     ttot_loss += loss.item()
    los.append(tot_loss)
    # tlos.append(ttot_loss)
    # print(tot_loss, ttot_loss)

  0%|          | 0/205 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/torch/autograd/__init__.py:197: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646703877/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 205/205 [00:10<00:00, 18.93it/s]


128.48864296078682


100%|██████████| 205/205 [00:06<00:00, 33.18it/s]


94.26303192973137


100%|██████████| 205/205 [00:06<00:00, 33.43it/s]


77.67878220416605


 78%|███████▊  | 160/205 [00:04<00:01, 33.53it/s]

In [ ]:

plt.plot(los, label="train")
plt.plot(tlos, label="test")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
net.eval()
all_loss_train = []
with torch.no_grad():
    tot_loss = 0
    for i, (inputs, labels) in enumerate(tqdm(train_loader), 0):
        l = [m.outer(m) for m in inputs]
        inputs = torch.empty((batch_size,)+l[0].shape, dtype=torch.float32)
        for j in range(len(l)):
            inputs[j] = l[j]
        inputs = torch.Tensor(inputs).unsqueeze(1) #.to_sparse()
        # inputs = torch.Tensor(inputs).unsqueeze(1) #.to_sparse()
        outputs = net(inputs).ravel()
        all_loss_train.extend((abs(outputs - labels) / abs(labels)).tolist())
        

In [ ]:
print(test.x.shape)
print(test.y.shape)

In [ ]:
439%5

In [ ]:
all_loss_test = []
with torch.no_grad():
    tot_loss = 0
    for i, (inputs, labels) in enumerate(tqdm(test_loader), 0):
        # labels = labels.reshape((-1, 1))
        # inputs = torch.Tensor(inputs).unsqueeze(1) #.to_sparse()
        l = [m.outer(m) for m in inputs]
        size = len(l)
        inputs = torch.empty((size,)+l[0].shape, dtype=torch.float32)
        print(inputs.shape, labels.shape)
        for j in range(len(l)):
            inputs[j] = l[j]
        inputs = torch.Tensor(inputs).unsqueeze(1)
        outputs = net(inputs).ravel()
        # if outputs.shape != labels.shape:
        #     print(outputs)
        #     print(labels)
        all_loss_test.extend((abs(outputs - labels) / abs(labels)).tolist())
    

In [ ]:
all_loss_train

In [ ]:
all_loss_train = np.array(all_loss_train)
all_loss_test = np.array(all_loss_test)


In [ ]:
len(np.where(all_loss_train < 1)[0]) / len(all_loss_train), len(np.where(all_loss_test < 1)[0]) / len(all_loss_test)

In [ ]:
good_pred_train = all_loss_train[np.where(all_loss_train < 1)]
good_pred_test = all_loss_test[np.where(all_loss_test < 1)]
good_pred_train.mean(), good_pred_train.std(), good_pred_test.mean(), good_pred_test.std()

In [ ]:
plt.hist(good_pred_train*100, bins=100, label="train")
plt.hist(good_pred_test*100, bins=100, label="test")
plt.legend()
plt.show()

In [ ]:
# all_loss = np.array(all_loss) * train.y_std + train.y_mean

In [ ]:
# all_loss